In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from playwright.async_api import async_playwright
import re
from random import randint
import time

In [2]:
from requests_cache import CachedSession, FileCache
session = CachedSession('http_cache', backend='filesystem')

#### Import the file with author links

In [3]:
all_links = pd.read_csv('goodreads-books.csv')

#### Loop through to get all authors' place of birth and genres

In [16]:
all_results = []
for index, row in all_links.iterrows(): 
    author_url = row['author_link']
    if (index + 1) % 20 == 0:
        time.sleep(29)
    else :
        time.sleep(randint(2, 6))
    try :
        cookies = {
            'ccsid': '742-3219006-7768950',
            '__qca': 'P0-2056977897-1700597220525',
            'lc-main': 'en_US',
            'ubid-main': '134-8959105-9465632',
            'csm-sid': '422-4199468-7443324',
            'session-id': '139-0527874-5342610',
            'session-id-time': '2332444182l',
            'session-token': '"YcXW7Jrp9Kdv2xKa/QBbi8yUBiSStEeiP28qLMGK2lSyGpW6Vw3XnxyZlwlCWSaBI23UAKLGg5mMq4mmjTTX5UMcJBP/l90fjKpsLaGzakLGDAaOiwusNYn+vfgZ/Ji7CiW7VI6WTJbTiBF+ktX/yjSNQ4si9UzZ1SHgTOg64mrm1HOLO7vvPVAkczr7/NrdNYMHm4GLcxETUjmEUyJHaBaWfvKLhflQHAYJK3X7Cvz7eQW7mrxpR5g/Z2bjmenos4nRiXFbFJ2gH5DpWR7mKNSM+yjI6VqdBq7LM4IoeVtaDzqpdTAlPHpDjMVc2EgfFeYGBLNWmZ6+Om+otBHfy9UZ+0k8Y+aI7yCqsEDM+7M="',
            'x-main': '"tlB5YO@Zn9Be1JOEOemZ3P13Hr@omAAwbvIMT7FJfMrlisFJpaLbLMo0vaTC0e9O"',
            'at-main': 'Atza|IwEBIDcy3eo7a_YNdQf_8OnGzwcey-G5RVHK7XfP5rWXif-4_Vi35NDoDynNj1XETaFcU4fwz2LR4aNWa7pNu4JaaOJyD3hWjg0TQM99HqHcdmQ7J8LLEkzbaY0Mr-w5fpbHcfNa5ex7oXPhsgAkO1SaBjRcwSW8Kn2LTONWfK_HCzEMd6yO04_V-rxxEDKo4CGs5f_qYC8N64UPEU5z8ujeLDuQR1yMOJq7x21AcB9nK0KEwMR1SFKqz8L-v6i6x8tF9PB5m8pC5ucLxADBO0u7Z7OV',
            'sess-at-main': '"ZgjCrkknePmuQiQKH0K2aBDhKl0Y1RS0IzdZ4GiPYrs="',
            'likely_has_account': 'true',
            'locale': 'en',
            '_session_id2': '3f6585ce11f778d9ddd2b236a4e40a86',
            '__gads': 'ID=ecab0ffeaa3f37cd:T=1701651617:RT=1702340873:S=ALNI_MaWm8wnJxWWL6w4EtAJDBKW8Mzakw',
            '__gpi': 'UID=00000ce2bab78f39:T=1701651617:RT=1702340873:S=ALNI_MbP8LYnaonBenczWFditmUQm_kvgA',
        }

        headers = {
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
            'Accept-Language': 'en-US,en;q=0.9',
            'Cache-Control': 'max-age=0',
            'Connection': 'keep-alive',
            # 'Cookie': 'ccsid=742-3219006-7768950; __qca=P0-2056977897-1700597220525; lc-main=en_US; ubid-main=134-8959105-9465632; csm-sid=422-4199468-7443324; session-id=139-0527874-5342610; session-id-time=2332444182l; session-token="YcXW7Jrp9Kdv2xKa/QBbi8yUBiSStEeiP28qLMGK2lSyGpW6Vw3XnxyZlwlCWSaBI23UAKLGg5mMq4mmjTTX5UMcJBP/l90fjKpsLaGzakLGDAaOiwusNYn+vfgZ/Ji7CiW7VI6WTJbTiBF+ktX/yjSNQ4si9UzZ1SHgTOg64mrm1HOLO7vvPVAkczr7/NrdNYMHm4GLcxETUjmEUyJHaBaWfvKLhflQHAYJK3X7Cvz7eQW7mrxpR5g/Z2bjmenos4nRiXFbFJ2gH5DpWR7mKNSM+yjI6VqdBq7LM4IoeVtaDzqpdTAlPHpDjMVc2EgfFeYGBLNWmZ6+Om+otBHfy9UZ+0k8Y+aI7yCqsEDM+7M="; x-main="tlB5YO@Zn9Be1JOEOemZ3P13Hr@omAAwbvIMT7FJfMrlisFJpaLbLMo0vaTC0e9O"; at-main=Atza|IwEBIDcy3eo7a_YNdQf_8OnGzwcey-G5RVHK7XfP5rWXif-4_Vi35NDoDynNj1XETaFcU4fwz2LR4aNWa7pNu4JaaOJyD3hWjg0TQM99HqHcdmQ7J8LLEkzbaY0Mr-w5fpbHcfNa5ex7oXPhsgAkO1SaBjRcwSW8Kn2LTONWfK_HCzEMd6yO04_V-rxxEDKo4CGs5f_qYC8N64UPEU5z8ujeLDuQR1yMOJq7x21AcB9nK0KEwMR1SFKqz8L-v6i6x8tF9PB5m8pC5ucLxADBO0u7Z7OV; sess-at-main="ZgjCrkknePmuQiQKH0K2aBDhKl0Y1RS0IzdZ4GiPYrs="; likely_has_account=true; locale=en; _session_id2=3f6585ce11f778d9ddd2b236a4e40a86; __gads=ID=ecab0ffeaa3f37cd:T=1701651617:RT=1702340873:S=ALNI_MaWm8wnJxWWL6w4EtAJDBKW8Mzakw; __gpi=UID=00000ce2bab78f39:T=1701651617:RT=1702340873:S=ALNI_MbP8LYnaonBenczWFditmUQm_kvgA',
            'If-None-Match': 'W/"31c7ac917838403d6c1869a7dbadecf5"',
            'Sec-Fetch-Dest': 'document',
            'Sec-Fetch-Mode': 'navigate',
            'Sec-Fetch-Site': 'none',
            'Sec-Fetch-User': '?1',
            'Upgrade-Insecure-Requests': '1',
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'sec-ch-ua': '"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-platform': '"macOS"',
        }

        response = session.get(f"{author_url}", cookies=cookies, headers=headers)
        print(f"scraping {author_url}")
        soup_doc = BeautifulSoup(response.text)

        # now search for first author of each book and keep only the book that matches the first occurence of the correct author
        place_of_birth_element = soup_doc.select_one('.dataTitle')
        place_of_birth = place_of_birth_element.next_sibling if place_of_birth_element else 'Not available'

        genres_element = soup_doc.select_one('.dataItem')  # Adjust the selector if needed
        author_genres = genres_element.text if genres_element else 'Not available'


        # put it all in a dictionary
        this_author = {
            'gr_link' : author_url,
            'author_place' : place_of_birth,
            'author_genres' :author_genres
            }
        all_results.append(this_author)  
    except Exception as e:
        print("Failed with", e)
        # If it fails, continue to the next entry
        continue

scraping https://www.goodreads.com/author/show/5261819.Marc_Cameron?from_search=true&from_srp=true
scraping https://www.goodreads.com/author/show/4178.Cormac_McCarthy?from_search=true&from_srp=true
scraping https://www.goodreads.com/author/show/14255.Danielle_Steel?from_search=true&from_srp=true
scraping https://www.goodreads.com/author/show/22458670.Ana_Reyes?from_search=true&from_srp=true
scraping https://www.goodreads.com/author/show/21098177.Ali_Hazelwood?from_search=true&from_srp=true
scraping https://www.goodreads.com/author/show/3780.James_Patterson?from_search=true&from_srp=true
scraping https://www.goodreads.com/author/show/4575289.Leigh_Bardugo?from_search=true&from_srp=true
scraping https://www.goodreads.com/author/show/8103667.Lucy_Score?from_search=true&from_srp=true
scraping https://www.goodreads.com/author/show/12577.Douglas_Preston?from_search=true&from_srp=true
scraping https://www.goodreads.com/author/show/4826394.Grady_Hendrix?from_search=true&from_srp=true
scraping 

In [18]:
all_authors = pd.DataFrame(all_results)
all_authors.to_csv('aithors_info.csv')